In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import regex
import re

# Import CRUD module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 30670
db = 'AAC'
col = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db, col)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

fig = px.pie(df, names='breed', title="All Animals", hole=0.2)
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Page layout
app.layout = html.Div([
# html.Div(id='hidden-div', style={'display':'none'}),
    # Title
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    
    # Grazioso Salvare Logo
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    
    # Unique identifier
    html.Center(html.B(html.H2('Project Two - Jacob Senior'))),
    html.Hr(),
    html.Br(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content': 'center', 'align-items': 'center'},
             children=[
        dcc.Graph(
            id='graph-id',
            figure=fig
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    html.Hr(),
    
    # Dropdown for resuce filtering
    html.Div(
        dcc.Dropdown(id='dropdown-id',
                     options=['None', 'Water', 'Mountain or Wilderness', 'Disaster or Individual Tracking'],
                     value='None',
                     searchable=False,
                     clearable=False
                    )
    ),
    
    # Animal data table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable="single",
                         selected_rows=[0],
                         selected_columns=[],
                         sort_action="native",
                         sort_mode="multi",
                         page_action="native",
                         page_current=0,
                         page_size=50,
                        ),
    html.Br()
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('dropdown-id', 'value')])
def update_dashboard(dropdown):

    # Update data table according to selected filter
    if (dropdown == 'Water'):
        # Create regex pattern to find accepted breeds (case insensitive)
        breeds = '(?i)(Labrador Retriever|Chesa Bay Retr|Newfoundland)'
        
        # Filter documents
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog',
                                                'breed' : { '$regex':breeds },
                                                'sex_upon_outcome' : 'Intact Female',
                                                'age_upon_outcome_in_weeks' : {'$gte': 26, '$lte': 156}
                                               }))
        
    elif (dropdown == 'Mountain or Wilderness'):
        # Create regex pattern to find accepted breeds (case insensitive)
        breeds = '(?i)(German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler)'
        
        # Filter documents
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog',
                                                'breed' : { '$regex':breeds },
                                                'sex_upon_outcome' : 'Intact Male',
                                                'age_upon_outcome_in_weeks' : {'$gte': 26, '$lte': 156}
                                               }))
        
    elif (dropdown == 'Disaster or Individual Tracking'):
        # Create regex pattern to find accepted breeds (case insensitive)
        breeds = '(?i)(Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler)'
        
        # Filter documents
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog',
                                                'breed' : { '$regex':breeds },
                                                'sex_upon_outcome' : 'Intact Male',
                                                'age_upon_outcome_in_weeks' : {'$gte': 20, '$lte': 300}
                                               }))
        
    else:
        # If none selected, return all documents
        df = pd.DataFrame.from_records(db.read({}))

    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('dropdown-id', 'value')]
)
def update_graphs(viewData, value):
    if viewData is None:
        return
    
    # Load pie chart data
    dff = pd.DataFrame.from_dict(viewData)
    
    # Update pie chart title
    if value == 'None':
        chartTitle = 'All Animals'
    else:
        chartTitle = value + ' Rescue Animals' 
        
    # Create pie chart figure
    figure = px.pie(dff, names='breed', title=chartTitle, hole=0.2)
    figure.update_traces(textposition='inside')
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return figure
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")],
)
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)

    # Because we only allow single row selection, the list can be converted to a row index here
    if not index:
        row = 0
    else:
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:10471/
